In [2]:
import numpy as np
import pandas as pd
from PIL import Image
import collections, math

import matplotlib.pyplot as plt
%matplotlib inline 

In [ ]:
#crop test images by detections_full_AGNOSTICnms.pkl 取同类中score最高的，按类的数量加权平均

RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

import pickle 
with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
    detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    
CONF_THRESH = 0.8
outputs = []
count = np.zeros(len(detections_full_AGNOSTICnms))

for im in range(len(detections_full_AGNOSTICnms)):
    if im%1000 == 0:
        print(im)
    detects_im = detections_full_AGNOSTICnms[im]
    score_max = np.max(detects_im[:,5:], axis=1)
    inds = np.argmax(detects_im[:,5:], axis=1)
    labels = [FISH_CLASSES[ind+1] for ind in inds]
    columns = ['box1', 'box2', 'box3', 'box4']
    columns.extend(FISH_CLASSES)
    detects_im_df = pd.DataFrame(detects_im, columns=columns)
    detects_im_df['max_cls'] = labels
    detects_im_df['max_score'] = score_max
    detects_im_df = detects_im_df[detects_im_df["max_score"]>=CONF_THRESH]  
    detects_im_df['Counts'] = detects_im_df.groupby(['max_cls'])['max_cls'].transform('count')
    idx = detects_im_df.groupby(['max_cls'])['max_score'].transform(max) == detects_im_df['max_score']
    detects_im_df = detects_im_df[idx]
    detects_im_array = detects_im_df[columns].as_matrix()
    count[im] = detects_im_array.shape[0]
    if detects_im_array.shape[0] == 0:
        ind = np.argmax(score_max)
        outputs_im = detects_im[[ind], :]
    else:
        outputs_im = detects_im_array
    outputs.append(outputs_im)

print(sum([outputs[i].shape[0] for i in range(len(outputs))]))

In [ ]:
count8 = count

In [ ]:
collections.Counter(count)

In [ ]:
np.where(count==2)[0]
#np.ndarray.tolist(np.where(count==2)[0][:10])

In [ ]:
diff = [(count5[i]-count9[i])>0 and count9[i]!=0 for i in range(4777)]
ims_diff = []
for i in range(1,2):
    if diff[i] == True:
        ims_diff.append(i)

ims_diff

In [ ]:
#visualize test image crop
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims]

for j in np.ndarray.tolist(np.where(count==0)[0][-10:]):
    dets = outputs[j]
    print(dets)
    im = Image.open("../RFCN/JPEGImages/"+test_files[j])
    im = np.asarray(im)
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.imshow(im, aspect='equal')
    for i in range(dets.shape[0]):
        bbox = dets[i, :4]
        score = np.amax(dets[i,4:])
        index = np.argmax(dets[i,4:])
        class_name = FISH_CLASSES[index]
        #if not (bbox[0] == 0 and bbox[1] == 0 and bbox[2] == 0 and bbox[3] == 0):
        ax.add_patch(plt.Rectangle((bbox[0], bbox[1]),
                          bbox[2] - bbox[0],
                          bbox[3] - bbox[1], fill=False,
                          edgecolor='red', linewidth=3.5))
        ax.text(bbox[0], bbox[1] - 2,
                '{:s} {:.3f}'.format(class_name, score),
                bbox=dict(facecolor='blue', alpha=0.5),
                fontsize=14, color='white')

    ax.set_title(('Image {} detections with '
                  'p({} | box) >= {:.1f}').format(j, class_name, CONF_THRESH),fontsize=14)
    plt.axis('off')
    plt.tight_layout()
    plt.draw()

In [17]:
#submission from detections_full_AGNOSTICnms.pkl 取同类中score最高的，按类的数量加权平均

RFCN_MODEL = 'resnet101_rfcn_ohem_iter_30000'
FISH_CLASSES = ['NoF', 'ALB', 'BET', 'DOL', 'LAG', 'OTHER', 'SHARK', 'YFT']

import pickle 
with open('../data/RFCN_detections/detections_full_AGNOSTICnms_'+RFCN_MODEL+'.pkl','rb') as f:
    detections_full_AGNOSTICnms = pickle.load(f, encoding='latin1') 
    
CONF_THRESH = 0.8
outputs = np.ndarray((len(detections_full_AGNOSTICnms), len(FISH_CLASSES)), dtype=np.float64)
count = np.zeros(len(detections_full_AGNOSTICnms))

#for im in range(len(detections_full_AGNOSTICnms)):
for im in range(2,3):
    if im%1000 == 0:
        print(im)
    detects_im = detections_full_AGNOSTICnms[im]
    score_max = np.max(detects_im[:,5:], axis=1)
    inds = np.argmax(detects_im[:,5:], axis=1)
    labels = [FISH_CLASSES[ind+1] for ind in inds]
    columns = ['box1', 'box2', 'box3', 'box4']
    columns.extend(FISH_CLASSES)
    detects_im_df = pd.DataFrame(detects_im, columns=columns)
    detects_im_df['max_cls'] = labels
    detects_im_df['max_score'] = score_max
    detects_im_df = detects_im_df[detects_im_df["max_score"]>=CONF_THRESH]
    print(detects_im_df)
    count[im] = detects_im_df.shape[0]
    detects_im_df['Counts'] = detects_im_df.groupby(['max_cls'])['max_cls'].transform('count')
    idx = detects_im_df.groupby(['max_cls'])['max_score'].transform(max) == detects_im_df['max_score']
    detects_im_df = detects_im_df[idx]
    l = FISH_CLASSES.copy()
    l.append('Counts')
    detects_im_array = detects_im_df[l].as_matrix()
    print(detects_im_array)
    if detects_im_array.shape[0] == 0:
        ind = np.argmax(score_max)
        outputs_im = detects_im[ind,4:12]
    else:
        outputs_im = np.average(detects_im_array[:,:-1], axis=0, weights=detects_im_array[:,-1], returned=False)
    outputs[im] = outputs_im
    


         box1        box2         box3        box4       NoF       ALB  \
0  604.372192   99.314407   923.525879  213.722458  0.000411  0.998497   
1  312.184143  127.184914   684.601257  265.319366  0.000080  0.997262   
2  927.494385  143.513062  1195.756836  268.516327  0.011148  0.985716   

        BET           DOL           LAG     OTHER         SHARK           YFT  \
0  0.000940  7.386075e-07  6.028753e-07  0.000148  6.061006e-07  2.904518e-06   
1  0.002547  2.058682e-07  4.018886e-07  0.000109  4.156222e-07  9.210435e-07   
2  0.000404  3.505660e-05  7.493963e-05  0.002543  9.936455e-06  6.904111e-05   

  max_cls  max_score  
0     ALB   0.998497  
1     ALB   0.997262  
2     ALB   0.985716  
[[  4.10640117e-04   9.98496771e-01   9.39608843e-04   7.38607469e-07
    6.02875275e-07   1.48173902e-04   6.06100627e-07   2.90451771e-06
    3.00000000e+00]]


In [18]:
count[2]

3.0

In [9]:
collections.Counter(count)

Counter({0.0: 682,
         1.0: 3241,
         2.0: 617,
         3.0: 143,
         4.0: 50,
         5.0: 24,
         6.0: 14,
         7.0: 2,
         8.0: 3,
         9.0: 1})

In [4]:
collections.Counter(count)

Counter({0.0: 636, 1.0: 4079, 2.0: 62})

In [14]:
# CLIP_THRESH = 0.02
# outputs_cliped = np.clip(outputs, CLIP_THRESH, 1, out=None)
# outputs_cliped = outputs_cliped/np.sum(outputs_cliped, axis=1, keepdims=True)
# test_preds = outputs_cliped[:1000]
# train_preds = outputs_cliped[1000:]

#temperature
T = 2.5
outputs_T = np.exp(np.log(outputs)/T)
outputs_T = outputs_T/np.sum(outputs_T, axis=1, keepdims=True)
test_preds = outputs_T[:1000]
train_preds = outputs_T[1000:]


with open("../RFCN/ImageSets/Main/test.txt","r") as f:
    ims = f.readlines()
test_files = [im[:-1]+'.jpg' for im in ims[:1000]]
train_files = [im[:-1] for im in ims[1000:]]
with open("../RFCN/ImageSets/Main/train_test.txt","r") as f:
    train_file_labels = f.readlines()

log_losses = []
for i in range(len(train_preds)):
    im = train_files[i]
    for im_label in train_file_labels:
        if im == im_label[:9]:
            label = im_label[10:-1]
            index = FISH_CLASSES.index(label)
            log_losses.append(-math.log(train_preds[i][index]))
log_loss = sum(log_losses) / float(len(log_losses))
print('logloss of train is', log_loss )

import datetime

submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
#submission.loc[:, 'image'] = pd.Series(test_files, index=submission.index)
submission.insert(0, 'image', test_files)

now = datetime.datetime.now()
# info = 'RFCN_AGONOSTICnms_clsMaxAve_resnet101_rfcn_ohem_iter_30000_conf{:.1f}_'.format(CONF_THRESH) + 'clip{:.2f}_'.format(CLIP_THRESH) + '{:.4f}'.format(log_loss)
info = 'RFCN_AGONOSTICnms_clsMaxAve_'+RFCN_MODEL+'_conf{:.2f}_T{}_'.format(CONF_THRESH, T) + '{:.4f}'.format(log_loss)
sub_file = 'submission_' + info + '.csv'
submission.to_csv(sub_file, index=False)

logloss of train is 0.11326009097947366


In [15]:
test_files

['img_00005.jpg',
 'img_00007.jpg',
 'img_00009.jpg',
 'img_00018.jpg',
 'img_00027.jpg',
 'img_00030.jpg',
 'img_00040.jpg',
 'img_00046.jpg',
 'img_00053.jpg',
 'img_00071.jpg',
 'img_00075.jpg',
 'img_00102.jpg',
 'img_00103.jpg',
 'img_00109.jpg',
 'img_00119.jpg',
 'img_00120.jpg',
 'img_00125.jpg',
 'img_00128.jpg',
 'img_00129.jpg',
 'img_00133.jpg',
 'img_00138.jpg',
 'img_00141.jpg',
 'img_00152.jpg',
 'img_00161.jpg',
 'img_00164.jpg',
 'img_00170.jpg',
 'img_00171.jpg',
 'img_00172.jpg',
 'img_00175.jpg',
 'img_00180.jpg',
 'img_00196.jpg',
 'img_00212.jpg',
 'img_00223.jpg',
 'img_00230.jpg',
 'img_00232.jpg',
 'img_00244.jpg',
 'img_00272.jpg',
 'img_00280.jpg',
 'img_00282.jpg',
 'img_00285.jpg',
 'img_00289.jpg',
 'img_00292.jpg',
 'img_00302.jpg',
 'img_00311.jpg',
 'img_00320.jpg',
 'img_00331.jpg',
 'img_00358.jpg',
 'img_00374.jpg',
 'img_00375.jpg',
 'img_00376.jpg',
 'img_00380.jpg',
 'img_00383.jpg',
 'img_00398.jpg',
 'img_00407.jpg',
 'img_00417.jpg',
 'img_0043